In [3]:
import numpy as np
from datetime import datetime, date, timedelta
import pandas as pd

In [191]:
class bonds:
        
    def __init__(self,
                 N : np.float,
                 tf : np.float,
                 tr : np.float,
                 d_emis : np.str,
                 d_jouis : np.str,
                 d_ech : np.str,
                 d_val : np.str):  
    
        self.N = N
        self.tf = tf
        self.tr = tr
        self.d_emis = datetime.strptime(d_emis, '%d-%m-%Y').date()
        self.d_jouis = datetime.strptime(d_jouis, '%d-%m-%Y').date()
        self.d_ech = datetime.strptime(d_ech, '%d-%m-%Y').date()
        self.d_val = datetime.strptime(d_val, '%d-%m-%Y').date()
        
        
        "calcul intermediaire"
        
        # maturite residuelle et initiale
        self.mr = (self.d_ech - self.d_val).days
        self.mi = (self.d_ech - self.d_emis).days
        
        # date du dernier coupon (ddc) et date du prochain coupon (dpc)
        ddc = self.d_jouis
        dpc = ddc.replace(year = ddc.year+1)
        while dpc < self.d_val:
            dpc = dpc.replace(year= dpc.year +1)
        self.ddc = dpc.replace(year = dpc.year-1)
        self.dpc = dpc
        
        # Nombre de jours exact de l’année écoulée ???????
        self.A = 365
        
        # nombre de jours entre date de valorisation et date du prochain coupon 
        self.nj = (self.dpc - self.d_val).days 
        
        # nombre de coupon restant avant la maturité
        self.n = (self.d_ech.year - self.dpc.year) + 1
        
        # Date du premier coupon ?????????
        # d_pc = "date_jouissnce+ frequence"???
        
           
        "sortie"

        self.dirty_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[0]
        self.coupon_couru = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[1]
        self.clean_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[2]
    
    
    def calculator(self,N,tf,tr,d_emis,d_jouis,d_ech,d_val): #7 parametres dont 4 dates

        if self.mi <= self.A : #maturite initiale inferieure a 1an

            C = N * tf * self.mi / 360
            P = (N + C) / (1+ tr*self.mr /360)
            Cc = N*tf* (self.mi-self.mr) / 360

        else:  #maturite initiale superieure a 1an

            if self.mr <= self.A: #maturite residuelle inferieure a 1an

                if d_jouis == d_emis: #Ligne simple, ou ligne atypique à plusieurs flux

                    C = N*tf
                    P = (N + C) / (1+ tr*self.mr/360)
                    Cc = N*tf* (self.d_val - self.ddc).days  / self.A 

                else: # Ligne atypique à un seul flux

                    C = N*tf*mi
                    P = (N + C) / (1+ tr*self.mr/360)
                    Cc = N*tf* (self.d_val-d_emis).days / self.A

            else: # maturite residuelle superieure a 1an

                if d_jouis == d_emis : # Ligne simple

                    C = N*tf
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    s = 0
                    for i in range(1,self.n+1): # n compris
                        s = s + C/ (1+tr)**(i-1)
                    P = ( s + N/ (1+tr)**(self.n-1))  / (1+tr)**(self.nj / self.A)

                else: # Ligne atypique 
                    """
                    if d_val < d_pc: # ligne a plusieurs flux où Date d’évaluation < Date du détachement du premier coupon

                        C1 = N*tf* (d_pc-d_emis) /A  # Premier coupon plein
                        C = N*tf  # Les autres coupons pleins
                        Cc = N*tf* (d_val-d_emis) / A
                        for i in range(2,n+1): # n compris
                            s = s + C1 + C/ (1+tr)**(i-1)
                        P = ( s + N/ (1+tr)**(n-1))  / (1+tr)**(nj/A)
                    
                    else: # Ligne atypique à 1 seul flux ou (à plusieurs flux où d_val >= d_pc)
                    """
                    C = N*tf
                    Cc = N*tf* (d_val-self.ddc) / A
                    s = 0
                    for i in range(1,self.n+1): #n compris
                        s = s + C/ (1+tr)**(i-1)
                    P = ( s + N/ (1+tr)**(self.n-1))  / (1+tr)**(self.nj / self.A)
                        
        return  (P,Cc,P-Cc)
            
            


In [213]:
N = 100000
tf = 0.039
tr = 0.02463
d_emis = "21-10-2014"
d_jouis = "21-10-2014"
d_ech = "21-10-2019"
d_val = "23-03-2018"
bond = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val)
bond.dirty_price

103824.75673488462

In [214]:
bond.coupon_couru

1634.7945205479452

In [215]:
bond.clean_price

102189.96221433667